### Script for coordinates conversion from MN03 (Swiss system) to WGS 84 (GPS system)

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import requests

In [71]:
#def swiss_coord_to_gps(x, y):
#   y_prime = (y - 600000)/1000000
#    x_prime = (x - 200000)/1000000
#    lambda_prime = 2.6779094 + 4.728982 * y_prime + 0.791484 * y_prime * x_prime + 0.1306 * y_prime * x_prime**2 - 0.0436 * y_prime**3
#    phi_prime = 16.9023892 + 3.238272 * x_prime - 0.270978 * y_prime**2 - 0.002528 * x_prime**2 - 0.0447 * y_prime**2 * x_prime - 0.0140 * x_prime**3
#    lon = lambda_prime * 100 / 36
#    lat = phi_prime * 100 / 36
#    return lon, lat

In [99]:
#def swiss_coord_to_gps_2(x, y):
#    Y = y/1000000 - 0.6
#    X = x/1000000 -0.2
#    a1 = 4.72973056 + 0.7925714*X + 0.132812*X**2 + 0.02550*X**3 + 0.0048*X**4 
#    a3 = - 0.044270 - 0.02550*X - 0.0096*X**2
#    a5 = 0.00096
#   p0 = 3.23864877*X - 0.0025486*X**2 - 0.013245*X**3 + 0.000048*X**4
#    p2 = -0.27135379 - 0.0450442*X - 0.007553*X**2 - 0.00146**3
#    p4 = 0.002442 + 0.00132*X
#    lambda_ = 2.67825 + a1*Y + a3*Y**3 + a5*Y**5
#    phi_ = 16.902866 + p0 + p2*Y**2 + p4*Y**4
#    return lambda_*100/36, phi_*100/36

#### Function base definition for request to api 'geodesy' 

In [2]:
def request_to_api(east, north, alt):
    re = requests.get(f'http://geodesy.geo.admin.ch/reframe/lv03towgs84?easting={east}&northing={north}&altitude={alt}&format=json')
    if re.status_code == 200:
        response = re.json()
        return response.get('northing'), response.get('easting'), response.get('altitude')
    else:
        return f'There is a problem with coordinates {east} , {north}'

Test of the request function :

In [3]:
request_to_api(688000, 293000, 550)

('47.78162339002335', '8.612821745531397', '596.3189703738317')

#### IMPORT (AND CLEAR) YOUR DATASET

In [11]:
data = pd.read_excel('DataFrames/data_parcelles_with_swiss_coord.xlsx')

In [12]:
data.drop('Unnamed: 0', axis=1, inplace=True)

In [13]:
data.head()

,PARCELLE,INVNR,X,Y,Z25,PRODREG,HT_VEG,DATE,SLOPE25,ASPECT25,...,V_LARCH,V_ARVEN,V_UENDH,V_BUCHE,V_AHORN,V_ESCHE,V_EICHE,V_CASTA,V_UELBH,LFI
0,51,150,688000,293000,669.6,1,3,1984-04-10,58.407726,67.342415,...,0.0,0.0,0.0,184.359112,0.00000,0.0,0.000000,0.0,0.0,LFI1
1,384,150,689000,288000,517.4,1,2,1984-04-09,55.683254,253.354935,...,0.0,0.0,0.0,230.021045,0.00000,0.0,0.000000,0.0,0.0,LFI1
2,1239,150,720000,281000,518.7,2,2,1985-04-01,43.496788,356.177185,...,0.0,0.0,0.0,322.259358,41.57716,0.0,0.000000,0.0,0.0,LFI1
3,1419,150,717000,280000,517.3,2,2,1985-03-27,29.557123,15.000126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LFI1
4,1431,150,723000,280000,493.6,2,2,1985-04-19,53.450974,57.142380,...,0.0,0.0,0.0,152.192812,0.00000,0.0,91.682308,0.0,0.0,LFI1


Order your dataset :

In [14]:
data.sort_values('PARCELLE', inplace=True)

### Loop of requests

In the case you have to request gps with just one request per line...

In [15]:
len_df = len(data)

In [16]:
list_of_id = data['PARCELLE'].unique()

Here we have to define numpy list for all our variables and our "id" :

In [17]:
IND_arr = data['PARCELLE'].to_numpy()
X_arr = data['X'].to_numpy()
Y_arr = data['Y'].to_numpy()
ALT_arr = data['Z25'].to_numpy()
lat_arr = np.empty(len_df)
lon_arr = np.empty(len_df)
alt_arr = np.empty(len_df)

Test : if I set an input of my id (here : 'PARCELLE'), what are the outputs ?... It should be all the index lines for this id...

In [18]:
np.where(IND_arr==51)

(array([0, 1, 2, 3], dtype=int64),)

Now, it can be launched <br>
De-comment the good lines, if you work with altitude or not.

In [19]:
for ind, id in enumerate(list_of_id):

    index_list = np.where(IND_arr==id)

    init = index_list[0][0]

    try:
        # lat, lon, _ = request_to_api(X_arr[init], Y_arr[init], 500) # --- code without altitude output
        lat, lon, alt = request_to_api(X_arr[init], Y_arr[init], ALT_arr[init]) # --- code wit altitude output
    except:
        print(f'Problem with index : {id} - on line {init} of the orginal dataset ...')

    for i in index_list[0]:
        lat_arr[i] = lat
        lon_arr[i] = lon
        alt_arr[i] = alt # --- code wit altitude output
    
    if ind%150==0:
        print(f'In process ... {ind} requests completed...')
    
data['X'] = lat_arr
data['Y'] = lon_arr
data['Z25'] = alt_arr # --- code wit altitude output
    

In process ... 0 requests completed...
In process ... 150 requests completed...
In process ... 300 requests completed...
In process ... 450 requests completed...
In process ... 600 requests completed...
In process ... 750 requests completed...
In process ... 900 requests completed...
In process ... 1050 requests completed...
In process ... 1200 requests completed...
In process ... 1350 requests completed...
In process ... 1500 requests completed...
In process ... 1650 requests completed...
In process ... 1800 requests completed...
In process ... 1950 requests completed...
In process ... 2100 requests completed...
In process ... 2250 requests completed...
In process ... 2400 requests completed...


In [20]:
data.describe(include='all')

C:\Users\PC\AppData\Local\Temp\ipykernel_6840\1336221646.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  data.describe(include='all')


,PARCELLE,INVNR,X,Y,Z25,PRODREG,HT_VEG,DATE,SLOPE25,ASPECT25,...,V_LARCH,V_ARVEN,V_UENDH,V_BUCHE,V_AHORN,V_ESCHE,V_EICHE,V_CASTA,V_UELBH,LFI
count,9612.000000,9612.000000,9612.000000,9612.000000,9612.000000,9612.000000,9612.000000,9612,9612.000000,9612.000000,...,9449.000000,9449.000000,9449.000000,9449.000000,9449.000000,9449.000000,9449.000000,9449.000000,9449.000000,9612
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2349,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1995-06-20 00:00:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LFI1
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2403
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1983-03-15 00:00:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-03 00:00:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,98928.478568,300.000000,46.647129,8.373582,1239.465745,3.672077,3.835622,NaN,59.484392,185.875246,...,28.301673,2.536589,0.400941,54.702979,9.177937,8.636591,2.782231,7.901297,11.157875,NaN
std,46104.772059,111.809215,0.402619,0.954026,412.388186,1.099672,1.154413,NaN,22.805324,109.322870,...,81.413670,18.299404,6.737838,120.521178,28.438937,35.339167,17.151467,42.815613,32.191319,NaN
min,51.000000,150.000000,45.860080,6.241239,331.652193,1.000000,2.000000,NaN,0.559998,0.079772,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,59317.000000,225.000000,46.294223,7.490960,920.655711,3.000000,3.000000,NaN,43.593285,89.030762,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN


In [21]:
data.rename(columns={"X":"LAT","Y":"LON", "Z25":"ALT"}, inplace=True)

Test with plotly_scatter_mapbox :

In [22]:
fig = px.scatter_mapbox(data, lat='LAT', lon='LON', mapbox_style='stamen-terrain')
fig.show()

### Final export of the dataframe :

In [23]:
data.to_excel('./Results/data_parcelles_with_gps.xlsx')